## Game of life using dask

In [11]:
import dask
import numpy as np
from dask.distributed import Client
import dask.array as da
import dask.multiprocessing


In [28]:
client = Client(n_workers=6, threads_per_worker=4, processes=True, memory_limit='2.5GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 15.75 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:63568,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 15.75 GiB
Comm: tcp://127.0.0.1:63601,Total threads: 3
Dashboard: http://127.0.0.1:63603/status,Memory: 3.94 GiB
Nanny: tcp://127.0.0.1:63573,


In [14]:
def read_input_file(input_file, chunk_size):
    # initialize board
    with open(input_file) as f:
        w, h = [int(x) for x in next(f).split()]
        # use the smallest data type
        board = np.zeros((w, h),dtype=np.uint8)
        for line_count, line in enumerate(f, start=0):
            single_numbers = line.split()
            x, y = map(int, single_numbers[:2])
            board[x][y] = 1
        dask_board = da.from_array(board, chunks=(chunk_size, chunk_size))
    return dask_board

In [20]:
def neighbors_number(board, row, col):
    # add plus two to create a proper iteration
    neighbors = board[max(0, row-1):min(board.shape[0], row+2), max(0, col-1):min(board.shape[1], col+2)]
    return np.sum(neighbors) - board[row, col]

def tick(board):
    # the copy takes a lot of space
    w, h = board.shape
    new_board = np.zeros((w, h),dtype=np.uint8)
    for r in range(w):
        for c in range(h):
            #for dead cell
            if board[r][c] == 0 and neighbors_number(board, r, c) == 3:
                new_board[r][c] = 1
            #for alive cell
            if board[r][c] == 1 and (neighbors_number(board, r, c) < 2 or neighbors_number(board, r, c) > 3):
                new_board[r][c] = 0
    return new_board


    

def play_game(dask_board, iterations):
    for i in range(iterations):
        dask_board = dask_board.map_overlap(tick, depth=1, boundary='none')
    final_board = dask_board.compute()
    return final_board


In [29]:
board = read_input_file('input_data/input.txt', 100)
print(play_game(board, 4))


2023-10-20 20:21:50,554 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [32]:
client.shutdown()

2023-10-20 20:34:11,004 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
